<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/TweetEval_EmotionTweetsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import re
import requests
import sklearn.metrics

In [3]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt"
VOCAB_SIZE=2000

In [4]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

2842

In [5]:
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))

In [6]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  3257 , Validation set :  374 , Test set :  1421


In [7]:
for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])
    tweets[i]=re.sub('#+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])
    val_tweets[i]=re.sub('#+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])
    test_tweets[i]=re.sub('#+', '', test_tweets[i])

In [8]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

In [9]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

In [10]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
train_df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [11]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=train_df,
                                                                   text_column = 'TWEET',
                                                                   label_columns = 'CATEGORY',
                                                                   val_df = val_df,
                                                                   maxlen = 65,
                                                                   preprocess_mode = 'bert')

['CATEGORY_0', 'CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3']
   CATEGORY_0  CATEGORY_1  CATEGORY_2  CATEGORY_3
1         0.0         0.0         1.0         0.0
2         1.0         0.0         0.0         0.0
3         0.0         1.0         0.0         0.0
4         1.0         0.0         0.0         0.0
5         0.0         0.0         0.0         1.0
['CATEGORY_0', 'CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3']
   CATEGORY_0  CATEGORY_1  CATEGORY_2  CATEGORY_3
1         1.0         0.0         0.0         0.0
2         1.0         0.0         0.0         0.0
3         1.0         0.0         0.0         0.0
4         1.0         0.0         0.0         0.0
5         1.0         0.0         0.0         0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [12]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 65
done.


In [13]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 3)

In [14]:
learner.fit_onecycle(lr = 2e-5, epochs = 3)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1086/1086 [==============================] - 358s 314ms/step - loss: 1.2039 - accuracy: 0.4565 - val_loss: 0.6138 - val_accuracy: 0.7594
Epoch 2/3
1086/1086 [==============================] - 339s 312ms/step - loss: 0.5032 - accuracy: 0.8196 - val_loss: 0.5375 - val_accuracy: 0.8102
Epoch 3/3
1086/1086 [==============================] - 338s 312ms/step - loss: 0.2016 - accuracy: 0.9331 - val_loss: 0.6464 - val_accuracy: 0.7995


In [15]:
data = test_df['TWEET'].tolist()

In [16]:
bert_pred=predictor.predict(data)

In [17]:
bert_pred[:10]

['CATEGORY_3',
 'CATEGORY_0',
 'CATEGORY_3',
 'CATEGORY_1',
 'CATEGORY_1',
 'CATEGORY_0',
 'CATEGORY_3',
 'CATEGORY_3',
 'CATEGORY_1',
 'CATEGORY_0']

In [18]:
y_true=np.array(test_df['CATEGORY'].tolist())

In [19]:
y_true[:10]

array([3, 0, 3, 1, 1, 0, 3, 3, 3, 0])

In [20]:
res_bert=[]
for i in range(len(bert_pred)):
  if bert_pred[i]=='CATEGORY_0':
    res_bert.append(0)
  elif bert_pred[i]=='CATEGORY_1':
    res_bert.append(1)
  elif bert_pred[i]=='CATEGORY_2':
    res_bert.append(2)
  else:
    res_bert.append(3)

In [21]:
res_bert=np.array(res_bert)
np.mean(res_bert==y_true)

0.8106966924700915

In [22]:
sklearn.metrics.f1_score(y_true, res_bert, average='macro')

0.7817310943952105

In [23]:
# 75.94 in 1 batchsize=6
# 77.29 in 2 batchsize=5